In [1]:
import rnavigate as rnav


# Combining PAIR-MaP files for multi-amplicon RNA: ".bp" files

PairMapper calculates correlated chemical events from DMS-MaP data analyzed
with ShapeMapper2. ShapeMapper2 can combine raw sequencing data from a multiple
amplicon experiment thoughtfully to create a single profile. PairMapper is
capable of analyzing this data. However, we were concerned that the areas of
overlap, and the disparity of read depth between amplicons and overlapping
regions could produce artifactual PAIRs that could bias structure prediction.

Here, we ".bp" files from amplicons that were analyzed individually in both
ShapeMapper2 and PairMapper. These files contain PAIRs-derived base-pairing
bonuses for input into structure prediction software.

Algorithm:

- PairMapper data from each amplicon is first reindexed to the full mRNA sequence using RNAvigate alignment features.
- Aligned PairMapper data is compared between overlapping amplicons.
  - Pairs occuring in 1 amplicon are kept.
  - If a PAIR occurs in both amplicons, the PAIR with the larger pairing bonus (more negative) is kept.
- The new data set is written to a new file with suffix "-combined.bp"

Data sets for which this was performed:

1. No tethering hairpin (TH) control (MCS) for both replicates
2. TH inserted into 5'UTR
3. TH inserted into 5' portion of coding sequence
4. TH inserted into 3' portion of coding sequence
5. TH inserted into 3'UTR

In [2]:
def merge_amplicons(full, amp1, amp2, amp3):
    p1 = amp1.data["bp_bonus"]
    p2 = amp2.data["bp_bonus"]
    p3 = amp3.data["bp_bonus"]
    full_seq = full.data["fasta"]

    columns = [col for col in p1.data.columns if col not in ["i", "j"]]

    pm_amplicons = [p1, p2, p3]
    for pm in pm_amplicons:
        pm.filter(fit_to=full_seq)
        pm.sequence = full_seq.sequence
        pm.data["i"] = pm.data["i_offset"]
        pm.data["j"] = pm.data["j_offset"]
        pm.data.sort_values(["i", "j"], inplace=True)
        if pm != p1:
            p1.data = p1.data.merge(pm.data, how="outer", on=["i", "j"], suffixes=["", "_y"], indicator=True)
            for idx, row in p1.data.iterrows():
                if row["_merge"] == "right_only" or ((row["_merge"] == "both") and (row["bonus"] > row["bonus_y"])):
                    for col in columns:
                        p1.data.loc[idx, col] = row[col+"_y"]
            p1.data = p1.data[["i", "j"] + columns]
    amp1.data["bp_bonus"].data = p1.data
    amp1.data["bp_bonus"].sequence = full_seq.sequence


## MCS replicate 1

In [3]:

kwargs = {'sep': '\s',
          'read_csv_kw': {'header':0, 'names':['i', 'j', 'bonus']},
          'default_metric': 'bonus',
          'fill':{'bonus':0}, 'cmaps':{'bonus':'viridis'}, 'mins_maxes':{'bonus':[-2,0]}}
amp1 = rnav.Sample(dmsmap="./MCS/seperate/MCS-Rep1-Primer1_MCS_primer1_profile.txt")
amp1.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./MCS/seperate/MCS-Rep1-Primer1.bp",
    sequence=amp1.data["dmsmap"].sequence, **kwargs)
amp2 = rnav.Sample(dmsmap="./MCS/seperate/MCS-Rep1-Primer2_MCS_primer2_profile.txt")
amp2.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./MCS/seperate/MCS-Rep1-Primer2.bp",
    sequence=amp2.data["dmsmap"].sequence, **kwargs)
amp3 = rnav.Sample(dmsmap="./MCS/seperate/MCS-Rep1-Primer3_MCS_primer3_profile.txt")
amp3.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./MCS/seperate/MCS-Rep1-Primer3.bp",
    sequence=amp3.data["dmsmap"].sequence, **kwargs)
full = rnav.Sample(
    fasta="./MCS/MCS.fa")
samples = [full, amp1, amp2, amp3]

merge_amplicons(*samples)

csv = amp1.data["bp_bonus"].data.to_csv(header=True, sep=' ', index=False, line_terminator='\n')
with open("MCS_rep1_combined.bp", 'w') as out_file:
    out_file.write('; ')
    out_file.write(csv)


C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the '

## MCS replicate 2

In [4]:
kwargs = {'sep': '\s',
          'read_csv_kw': {'header':0, 'names':['i', 'j', 'bonus']},
          'default_metric': 'bonus',
          'fill':{'bonus':0}, 'cmaps':{'bonus':'viridis'}, 'mins_maxes':{'bonus':[-2,0]}}
amp1 = rnav.Sample(dmsmap="./MCS/seperate/MCS-Rep2-Primer1_MCS_primer1_profile.txt")
amp1.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./MCS/seperate/MCS-Rep2-Primer1.bp",
    sequence=amp1.data["dmsmap"].sequence, **kwargs)
amp2 = rnav.Sample(dmsmap="./MCS/seperate/MCS-Rep2-Primer2_MCS_primer2_profile.txt")
amp2.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./MCS/seperate/MCS-Rep2-Primer2.bp",
    sequence=amp2.data["dmsmap"].sequence, **kwargs)
amp3 = rnav.Sample(dmsmap="./MCS/seperate/MCS-Rep2-Primer3_MCS_primer3_profile.txt")
amp3.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./MCS/seperate/MCS-Rep2-Primer3.bp",
    sequence=amp3.data["dmsmap"].sequence, **kwargs)
full = rnav.Sample(
    sample="MCS rep 1: full sequence",
    fasta="./MCS/MCS.fa",
    dmsmap="./MCS/MCS_Rep2_MCS_profile.txt",
    pairmap="./MCS/MCS-Rep2-pairmap.txt")
samples = [full, amp1, amp2, amp3]

merge_amplicons(*samples)

csv = amp1.data["bp_bonus"].data.to_csv(header=True, sep=' ', index=False, line_terminator='\n')
with open("MCS_rep2_combined.bp", 'w') as out_file:
    out_file.write('; ')
    out_file.write(csv)


C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the '

## 5UTR

In [5]:
kwargs = {'sep': '\s',
          'read_csv_kw': {'header':0, 'names':['i', 'j', 'bonus']},
          'default_metric': 'bonus',
          'fill':{'bonus':0}, 'cmaps':{'bonus':'viridis'}, 'mins_maxes':{'bonus':[-2,0]}}
amp1 = rnav.Sample(dmsmap="./5UTR/seperate/5UTR_primer1_5UTR_primer1_profile.txt")
amp1.data["bp_bonus"] = rnav.data.Interactions(
    filepath='./5UTR/seperate/5UTR_primer1.bp',
    sequence=amp1.data["dmsmap"].sequence, **kwargs)
amp2 = rnav.Sample(dmsmap="./5UTR/seperate/5UTR_primer2_5UTR_primer2_profile.txt")
amp2.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./5UTR/seperate/5UTR_primer2.bp",
    sequence=amp2.data["dmsmap"].sequence, **kwargs)
amp3 = rnav.Sample(dmsmap="./5UTR/seperate/5UTR_primer3_5UTR_primer3_profile.txt")
amp3.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./5UTR/seperate/5UTR_primer3.bp",
    sequence=amp3.data["dmsmap"].sequence, **kwargs)
full = rnav.Sample(fasta="./5UTR/5UTR.fa")
samples = [full, amp1, amp2, amp3]

merge_amplicons(*samples)

csv = amp1.data["bp_bonus"].data.to_csv(header=True, sep=' ', index=False, line_terminator='\n')
with open("5UTR_combined.bp", 'w') as out_file:
    out_file.write('; ')
    out_file.write(csv)


C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the '

## 3'UTR

In [6]:
kwargs = {'sep': '\s',
          'read_csv_kw': {'header':0, 'names':['i', 'j', 'bonus']},
          'default_metric': 'bonus',
          'fill':{'bonus':0}, 'cmaps':{'bonus':'viridis'}, 'mins_maxes':{'bonus':[-2,0]}}
amp1 = rnav.Sample(dmsmap="./3UTR/seperate/3UTR_primer1_3UTR_primer1_profile.txt")
amp1.data["bp_bonus"] = rnav.data.Interactions(
    filepath='./3UTR/seperate/3UTR_primer1.bp',
    sequence=amp1.data["dmsmap"].sequence, **kwargs)
amp2 = rnav.Sample(dmsmap="./3UTR/seperate/3UTR_primer2_3UTR_primer2_profile.txt")
amp2.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./3UTR/seperate/3UTR_primer2.bp",
    sequence=amp2.data["dmsmap"].sequence, **kwargs)
amp3 = rnav.Sample(dmsmap="./3UTR/seperate/3UTR_primer3_3UTR_primer3_profile.txt")
amp3.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./3UTR/seperate/3UTR_primer3.bp",
    sequence=amp3.data["dmsmap"].sequence, **kwargs)
full = rnav.Sample(fasta="./3UTR/3UTR.fa")
samples = [full, amp1, amp2, amp3]

merge_amplicons(*samples)

csv = amp1.data["bp_bonus"].data.to_csv(header=True, sep=' ', index=False, line_terminator='\n')
with open("3UTR_combined.bp", 'w') as out_file:
    out_file.write('; ')
    out_file.write(csv)


C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the '

## CDS 1

In [7]:
kwargs = {'sep': '\s',
          'read_csv_kw': {'header':0, 'names':['i', 'j', 'bonus']},
          'default_metric': 'bonus',
          'fill':{'bonus':0}, 'cmaps':{'bonus':'viridis'}, 'mins_maxes':{'bonus':[-2,0]}}
amp1 = rnav.Sample(dmsmap="./CDS1/seperate/CDS1_primer1_CDS1_primer1_profile.txt")
amp1.data["bp_bonus"] = rnav.data.Interactions(
    filepath='./CDS1/seperate/CDS1_primer1.bp',
    sequence=amp1.data["dmsmap"].sequence, **kwargs)
amp2 = rnav.Sample(dmsmap="./CDS1/seperate/CDS1_primer2_CDS1_primer2_profile.txt")
amp2.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./CDS1/seperate/CDS1_primer2.bp",
    sequence=amp2.data["dmsmap"].sequence, **kwargs)
amp3 = rnav.Sample(dmsmap="./CDS1/seperate/CDS1_primer3_CDS1_primer3_profile.txt")
amp3.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./CDS1/seperate/CDS1_primer3.bp",
    sequence=amp3.data["dmsmap"].sequence, **kwargs)
full = rnav.Sample(fasta="./CDS1/CDS1.fa")
samples = [full, amp1, amp2, amp3]

merge_amplicons(*samples)

csv = amp1.data["bp_bonus"].data.to_csv(header=True, sep=' ', index=False, line_terminator='\n')
with open("CDS1_combined.bp", 'w') as out_file:
    out_file.write('; ')
    out_file.write(csv)


C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the '

## CDS 3

In [8]:
kwargs = {'sep': '\s',
          'read_csv_kw': {'header':0, 'names':['i', 'j', 'bonus']},
          'default_metric': 'bonus',
          'fill':{'bonus':0}, 'cmaps':{'bonus':'viridis'}, 'mins_maxes':{'bonus':[-2,0]}}
amp1 = rnav.Sample(dmsmap="./CDS3/seperate/CDS3_primer1_CDS3_primer1_profile.txt")
amp1.data["bp_bonus"] = rnav.data.Interactions(
    filepath='./CDS3/seperate/CDS3_primer1.bp',
    sequence=amp1.data["dmsmap"].sequence, **kwargs)
amp2 = rnav.Sample(dmsmap="./CDS3/seperate/CDS3_primer2_CDS3_primer2_profile.txt")
amp2.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./CDS3/seperate/CDS3_primer2.bp",
    sequence=amp2.data["dmsmap"].sequence, **kwargs)
amp3 = rnav.Sample(dmsmap="./CDS3/seperate/CDS3_primer3_CDS3_primer3_profile.txt")
amp3.data["bp_bonus"] = rnav.data.Interactions(
    filepath="./CDS3/seperate/CDS3_primer3.bp",
    sequence=amp3.data["dmsmap"].sequence, **kwargs)
full = rnav.Sample(fasta="./CDS3/CDS3.fa")
samples = [full, amp1, amp2, amp3]

merge_amplicons(*samples)

csv = amp1.data["bp_bonus"].data.to_csv(header=True, sep=' ', index=False, line_terminator='\n')
with open("CDS3_combined.bp", 'w') as out_file:
    out_file.write('; ')
    out_file.write(csv)


C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.data = pd.read_csv(filepath, sep=sep, **read_csv_kw)
C:\Users\psirv\OneDrive - University of North Carolina at Chapel Hill\GitHub\RNAvigate\rnavigate\data\interactions.py:70: ParserWarning: Falling back to the '